# RNN과 어텐션을 사용한 자연어 처리

먼저 몇 개의 모듈을 임포트한다. 맷플롯립 그림을 저장하는 함수를 준비한다.

In [1]:
# 공통 모듈 임포트
import os
import matplotlib.pyplot as plt

# 그림을 저장할 위치
PROJECT_ROOT_DIR = '.'
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, 'images')
os.makedirs(IMAGES_PATH, exist_ok=True)


def save_fig(fig_id, tight_layout=True, fig_extension='png', resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + '.' + fig_extension)
    print(f'그림 저장 {fig_id}')
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, dpi=resolution, format=fig_extension)

## Char-RNN을 사용해 셰익스피어 문제 생성하기

예를 들어, 0~14까지 시퀀스를 2개씩 이동하면서 길이가 5인 윈도우로 나누어 본다(가령,`[0, 1, 2, 3, 4]`, `[2, 3, 4, 5, 6]`, 등). 그다음 이를 섞고 입력(처음 네 개의 스텝)과 타깃(마지막 네 개의 스텝)으로 나눈다(즉, `[2, 3, 4, 5, 6]`를 `[[2, 3, 4, 5], [3, 4, 5, 6]]`로 나눈다). 그다음 입력/타깃 쌍 세 개로 구성된 배치를 만든다:

In [2]:
import tensorflow as tf

n_steps = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_steps, 2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_steps))
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))
dataset = dataset.batch(3).prefetch(tf.data.AUTOTUNE)
for index, (X_batch, Y_batch) in enumerate(dataset):
    print(f'{"_" * 20} Batch {index}\nX_batch\n{X_batch.numpy()}\n{"=" * 5}\nY_batch\n{Y_batch.numpy()}')

____________________ Batch 0
X_batch
[[ 4  5  6  7]
 [ 2  3  4  5]
 [10 11 12 13]]
=====
Y_batch
[[ 5  6  7  8]
 [ 3  4  5  6]
 [11 12 13 14]]
____________________ Batch 1
X_batch
[[ 6  7  8  9]
 [ 0  1  2  3]
 [ 8  9 10 11]]
=====
Y_batch
[[ 7  8  9 10]
 [ 1  2  3  4]
 [ 9 10 11 12]]


### 훈련 데이터셋 만들기

In [3]:
from tensorflow import keras

shakespeare_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [4]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [5]:
''.join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [6]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [7]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [8]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [9]:
max_id = len(tokenizer.word_index)  # 고유한 문자 개수
dataset_size = tokenizer.document_count  # 전체 문자 개수

In [10]:
import numpy as np

[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

### 순차 데이터셋을 나누는 방법

In [11]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

### 순차 데이터를 윈도 여러 개로 자르기

In [12]:
n_steps = 100
window_length = n_steps + 1  # 타깃 = 한 글자 앞선 입력
dataset = dataset.window(window_length, 1, drop_remainder=True)

In [13]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [14]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [15]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, max_id), Y_batch))

In [16]:
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [17]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


### Char-RNN 모델 만들고 훈련하기

**경고**: 다음 코드는 하드웨어에 따라 실행하는데 24시간이 걸릴 수 있다. GPU를 사용하면 1~2시간 정도 걸릴 수 있다.

**노트**: `GRU` 클래스는 다음 매개변수에서 기본값을 사용할 때에만 GPU를 사용한다: `activation`, `recurrent_activation`, `recurrent_dropout`, `unroll`, `use_bias` `reset_after`. 이 때문에 `recurrent_dropout=0.2`를 주석 처리했다.

In [18]:
model = keras.models.Sequential(
    [
        keras.layers.GRU(128, dropout=0.2, return_sequences=True, input_shape=[None, max_id]),  # recurrent_dropout=0.2
        keras.layers.GRU(128, dropout=0.2, return_sequences=True),  # recurrent_dropout=0.2
        keras.layers.TimeDistributed(keras.layers.Dense(max_id, 'softmax'))
    ]
)
model.compile('adam', 'sparse_categorical_crossentropy')
# history = model.fit(dataset, epochs=10)
history = model.fit(dataset.take(1), epochs=10)

Epoch 1/10
1/1 [==============================] - 13s 13s/step - loss: 3.6614
Epoch 2/10
1/1 [==============================] - 1s 1s/step - loss: 3.6446
Epoch 3/10
1/1 [==============================] - 1s 1s/step - loss: 3.6266
Epoch 4/10
1/1 [==============================] - 1s 1s/step - loss: 3.6045
Epoch 5/10
1/1 [==============================] - 1s 1s/step - loss: 3.5783
Epoch 6/10
1/1 [==============================] - 1s 1s/step - loss: 3.5493
Epoch 7/10
1/1 [==============================] - 1s 1s/step - loss: 3.5119
Epoch 8/10
1/1 [==============================] - 1s 1s/step - loss: 3.4662
Epoch 9/10
1/1 [==============================] - 1s 1s/step - loss: 3.3983
Epoch 10/10
1/1 [==============================] - 1s 1s/step - loss: 3.3220


### Char-RNN 모델 사용하기

In [19]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [20]:
X_new = preprocess(['How are yo'])
Y_pred = np.argmax(model(X_new), -1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]  # 1st sentence, last char

' '

### 가짜 셰익스피어 텍스트를 생성하기

In [21]:
tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], 40).numpy()

array([[1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 2, 1, 0, 0, 1]],
      dtype=int64)

In [22]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, 1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [23]:
next_char('How are yo')

'n'

In [24]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [25]:
print(complete_text('t', temperature=0.2))

tnekh e                                            


In [26]:
print(complete_text('t'))

t$cinssgk;nt?a'riha, :dyojuh?gk&tvihanro ykpo
o$ gk


In [27]:
print(complete_text('t', temperature=2))

t c!3',m dqztbrar'd!,blbbtjvopvo:ono!oiu ?-sa
a, vl


### 상대가 있는 RNN

In [46]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, max_id), Y_batch))
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [47]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, max_id), Y_batch))
dataset = dataset.prefetch(tf.data.AUTOTUNE)

**노트**: 여기에서도 GPU 가속을 위해 `recurrent_dropout=0.2`을 주석 처리한다.

In [48]:
model = keras.models.Sequential(
    [
        keras.layers.GRU(
            128, dropout=0.2, return_sequences=True, stateful=True, batch_input_shape=[batch_size, None, max_id]
        ),  # recurrent_dropout=0.2
        keras.layers.GRU(128, dropout=0.2, return_sequences=True, stateful=True),  # recurrent_dropout=0.2
        keras.layers.TimeDistributed(keras.layers.Dense(max_id, 'softmax'))
    ]
)

In [51]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [53]:
model.compile('adam', 'sparse_categorical_crossentropy')
# history = model.fit(dataset, epochs=50, callbacks=[ResetStatesCallback()])
history = model.fit(dataset.take(1), epochs=50, callbacks=[ResetStatesCallback()])

Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 1.8977
Epoch 2/50
1/1 [==============================] - 0s 46ms/step - loss: 1.8562
Epoch 3/50
1/1 [==============================] - 0s 36ms/step - loss: 1.8150
Epoch 4/50
1/1 [==============================] - 0s 39ms/step - loss: 1.8252
Epoch 5/50
1/1 [==============================] - 0s 39ms/step - loss: 1.8354
Epoch 6/50
1/1 [==============================] - 0s 39ms/step - loss: 1.8244
Epoch 7/50
1/1 [==============================] - 0s 35ms/step - loss: 1.7892
Epoch 8/50
1/1 [==============================] - 0s 37ms/step - loss: 1.7819
Epoch 9/50
1/1 [==============================] - 0s 36ms/step - loss: 1.7570
Epoch 10/50
1/1 [==============================] - 0s 38ms/step - loss: 1.7715
Epoch 11/50
1/1 [==============================] - 0s 35ms/step - loss: 1.7472
Epoch 12/50
1/1 [==============================] - 0s 40ms/step - loss: 1.7266
Epoch 13/50
1/1 [==============================] - 0s 42ms/step

모델에 다른 크기의 배치를 사용하려면 상태가 없는 복사본을 만들어야 한다. 드롭아웃은 훈련에만 사용되기 때문에 삭제한다:

In [76]:
stateless_model = keras.models.Sequential(
    [
        keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
        keras.layers.GRU(128, return_sequences=True),
        keras.layers.TimeDistributed(keras.layers.Dense(max_id, 'softmax'))
    ]
)